In [54]:
#import numpy as np
import os # will use later on for getting the datasaet as well as saving the model and output. 

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms#, datasets

# Image processing:
from PIL import Image, ImageCms

In [66]:
#THIS IS MY ATTEMPT AT TRANSFERING THE KERAS BASELINE INTO PYTORCH -- tensors have different values do the ones
# to be ignored for now!!!!
#from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
#from skimage.io import imsave
#from keras.preprocessing.image import img_to_array, load_img
#import numpy as np
# Get images
#image = img_to_array(load_img('domddcz.png'))
#image = np.array(image, dtype=float)
#X = rgb2lab(1.0/255*image)[:,:,0]
#Y = rgb2lab(1.0/255*image)[:,:,1:]
#print(X.shape)
# Transform the data to a tensor -- move from keras to pytorch.
#X = transforms.ToTensor()(X)
#Y = transforms.ToTensor()(Y)
#print(X.shape)
# Output colorizations
#cur = np.zeros((400, 400, 3))
#cur[:,:,0] = X[0][:,:,0]
#cur[:,:,1:] = output[0]
#imsave("ehhhhhhhhhhhhhhhhhhhh123.png", lab2rgb(cur))
#im = transforms.ToPILImage()(X).convert("L")
#im.save('test.jpg', "JPEG")
#print(X)

(224, 224)
torch.Size([1, 224, 224])


In [2]:
# Get images
# Open image and discard alpha channel which makes wheel round rather than square
img_arr = Image.open("domddcz.png").convert('RGB')
# create a greyscale image #img_gray = Image.open('woman.jpg').convert('L')

In [3]:
# Convert the picture into Lab colourspace
srgb_p = ImageCms.createProfile("sRGB")
lab_p  = ImageCms.createProfile("LAB")

rgb2lab = ImageCms.buildTransformFromOpenProfiles(srgb_p, lab_p, "RGB", "LAB")
Lab = ImageCms.applyTransform(img_arr, rgb2lab)

In [47]:
#Get the Lightness channel, a&b channels don't matter too much for now. Maybe add them together later on.
image_lightnesses = Lab.getchannel('L')
#image_a = Lab.getchannel('A')
#image_b = Lab.getchannel('B')
#a.save('wtf.png')

In [5]:
#Rescale the picture: --works
#Lab_test = Lab_test.view(-1, 3, 224, 224)

image_lightness = transforms.ToTensor()(image_lightnesses)
#Lab_test[:,:,0] # get the first channel --BROKEN
#image_ab = Lab_test[:,:,1:] #get the other 2 channels.

In [28]:
def weights_init(model):
    if type(model) in [nn.Conv2d, nn.Linear]:
        nn.init.xavier_normal_(model.weight.data)
        nn.init.constant_(model.bias.data, 0.1)

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            # conv1
            nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 2, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 64),
            # conv2
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 2, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 128),
            # conv3
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 2, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 256),
            # conv4
            nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 512),
            # conv5
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 2, dilation = 2),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 2, dilation = 2),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 2, dilation = 2),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 512),
            # conv6
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 2, dilation = 2),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 2, dilation = 2),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 2, dilation = 2),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 512),
            # conv7
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1, dilation = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1, dilation = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1, dilation = 1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 512),
            # conv8
            nn.ConvTranspose2d(in_channels = 512, out_channels = 256, kernel_size = 4, stride = 2, padding = 1, dilation = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1, dilation = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1, dilation = 1),
            nn.ReLU(),
            # conv8_313
            nn.Conv2d(in_channels = 256, out_channels = 224, kernel_size = 1, stride = 1,dilation = 1),
            #eltwise product
            # softmax
            #nn.Softmax(dim=1),
            # decoding
            #nn.Conv2d(in_channels = 313, out_channels = 2, kernel_size = 1, stride = 1)
        )
        self.apply(weights_init)

    def forward(self, gray_image):
        # Zhang doesn't use pooling. Up/down-sampling it is.
        # Max pooling over a (2, 2) window #x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        #x = x.view(-1, self.num_flat_features(x)) - interesting part
        
        #print('gray_image',gray_image.size())
        features=self.features(gray_image)
        gray_image=gray_image/0.38
        return gray_image

In [29]:
test = Net()

In [30]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss(reduction='none').cuda()
params = list(test.parameters())
optimizer = torch.optim.Adam(params, lr = 0.001) #args.learning_rate

In [31]:
print(image_lightness.shape)

torch.Size([1, 1, 224, 224])


In [15]:
# Train the models
#total_step = len(data_loader)
image_lightness = image_lightness.view(-1, 1, 224, 224) 
#image_lightness = image_lightness.unsqueeze(0).float().cuda()

for epoch in range(100): #range(args.num_epochs):
    #print(epoch)
    # Set mini-batch dataset   

    outputs = test(image_lightness)
    #outputs = test(image_lightness)#.log()
    #output=outputs[0].cpu().data.numpy()
    
    #out_max=np.argmax(output,axis=0)
    #print('set',set(out_max.flatten())) # Shows us which set are we working on right now

    test.zero_grad()

    #loss.backward()
    optimizer.step()
    print(epoch)
    
#save the current model for future use.
#torch.save(test.state_dict(), 'model-100.ckpt') #, i+

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [10]:
# get the result out and reshape it - DON't need that now for some reason.
#for epoch in range(100):
#    outputs = outputs.squeeze(0)#.float().cuda()
    #output = output.squeeze(0)

#print(outputs.shape)
#result = cpu_pred.data.numpy()
#array_res = np.reshape(cpu_pred, (224,224) )
#print(array_res)

NameError: name 'outputs' is not defined

In [32]:
#outputs = outputs.squeeze(0)
#test_output = outputs.squeeze(0)
#test_output = test_output.squeeze(0)
print(test_output.shape)

torch.Size([224, 224])


In [36]:
#cpu_pred.type(torch.Float32)
#print(im.size)
#print(outputs.size)
im = transforms.ToPILImage()(test_output)#.convert("L") #.cpu()
#im.show()
#print(im.size)

In [34]:
im.save('ugh.jpg', "JPEG")

In [330]:
#TRYING TO FOLLOW THE KERAS BASELINE AGAIN, HOPING IT WILL MAKE THE PICTURE RIGHT, instead of 3x224.

#cpu_pred *= 128
#Output colorizations
#cur = np.zeros((224, 224, 3))
#cur[:,:,0] = cpu_pred[0][:,:,0] #THIS PART IS BROKEN - LOOK FOR A FIX. "TOO MANY INDICES FOR ARRAY."

#cur[:,:,1:] = output[0]
#imsave("woman_100_img_result.png", lab2rgb(cur))
#imsave("woman_100_img_gray_version.png", rgb2gray(lab2rgb(cur)))